In [12]:
"hello world"

'hello world'

In [13]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [14]:
df = pd.read_csv("data.csv")

df.head()

,review,sentiment
0,Every great gangster movie has under-currents ...,positive
1,"I just saw this film last night, and I have to...",positive
2,This film is mildly entertaining if one neglec...,negative
3,Quentin Tarantino's partner in crime Roger Ava...,negative
4,I sat through this on TV hoping because of the...,negative


In [15]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [16]:
df = normalize_text(df)

df.sample(3)

,review,sentiment
421,watched film time never really liked it fan te...,negative
420,go open air cinema greek summer night usually ...,negative
409,although many audio recording great musician l...,positive


In [17]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [18]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [19]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [21]:
import dagshub
dagshub.init(repo_owner='T-dot-prog', repo_name='capstone_project', mlflow=True)


mlflow.set_tracking_uri(uri= "https://dagshub.com/T-dot-prog/capstone_project.mlflow")

mlflow.set_experiment(experiment_name= "logistic_model_experiment")

Initialized MLflow to track repo "T-dot-prog/capstone_project"

2025-07-03 19:58:28,795 - INFO - Initialized MLflow to track repo "T-dot-prog/capstone_project"


Repository T-dot-prog/capstone_project initialized!

2025-07-03 19:58:28,795 - INFO - Repository T-dot-prog/capstone_project initialized!


<Experiment: artifact_location='mlflow-artifacts:/bf7c9c2927fd4104b46d4e04e4fd0add', creation_time=1751550902880, experiment_id='0', last_update_time=1751550902880, lifecycle_stage='active', name='logistic_model_experiment', tags={}>

In [23]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        # Specify pos_label to match your string labels
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, pos_label="positive")
        recall = recall_score(y_test, y_pred, pos_label="positive")
        f1 = f1_score(y_test, y_pred, pos_label="positive")

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, artifact_path= "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-07-03 19:59:54,247 - INFO - Starting MLflow run...
2025-07-03 19:59:54,684 - INFO - Logging preprocessing parameters...
2025-07-03 19:59:55,986 - INFO - Initializing Logistic Regression model...
2025-07-03 19:59:55,986 - INFO - Fitting the model...
2025-07-03 19:59:56,026 - INFO - Model training complete.
2025-07-03 19:59:56,029 - INFO - Logging model parameters...
2025-07-03 19:59:56,412 - INFO - Making predictions...
2025-07-03 19:59:56,428 - INFO - Calculating evaluation metrics...
2025-07-03 19:59:56,502 - INFO - Logging evaluation metrics...
2025-07-03 19:59:58,128 - INFO - Saving and logging the model...
2025/07/03 20:00:13 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-07-03 20:00:13,195 - INFO - Model training and logging completed in 18.51 seconds.
2025-07-03 20:00:13,195 - INFO - Accuracy: 0.64
2025-07-03 20:00:13,195 - INFO - Precision: 0.6610169491525424
2025-07-